In [191]:
import numpy as np
import haversine as hvs
from sklearn.metrics.pairwise import haversine_distances
from math import radians, sin, cos, sqrt, atan2
import time

def optimized_haversine_distances(lonlat1, lonlat2):
    """
    Calculate the haversine distances between two sets of longitude-latitude coordinates.
    lonlat1: array-like, shape (n, 2)
        Array of longitude-latitude coordinates for the first set of points.
    lonlat2: array-like, shape (m, 2)
        Array of longitude-latitude coordinates for the second set of points.
    Returns:
    distances: ndarray, shape (n, m)
        Array of haversine distances between each pair of points.
    """
    lon1, lat1 = lonlat1[:, 0], lonlat1[:, 1]
    lon2, lat2 = lonlat2[:, 0], lonlat2[:, 1]

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin((dlat) / 2) ** 2 + np.cos(lon1) * np.cos(lon2) * np.sin((dlon) / 2) ** 2
    distances = 2 * np.arcsin(np.sqrt(a))

    return distances


def slow_haversine_distances(x, y):
        diff_lat = y[:,0] - x[0]
        diff_lon = y[:,1] - x[1]
        a = np.sin(diff_lat / 2) ** 2 + (
            np.cos(x[0]) * np.cos(y[:,0]) * np.sin(diff_lon / 2) ** 2
        )
        c = 2 * np.arcsin(np.sqrt(a))
        return c

# Generate random longitude-latitude coordinates
n = 10_000
m = 1
lonlat1 = np.random.uniform(low=-180, high=180, size=(n, 2))
lonlat2 = np.random.uniform(low=-180, high=180, size=(m, 2))
# lonlat1[:,0] +=180
# lonlat2[:,0] +=180
lonlat1[:,1] /= 2
lonlat2[:,1] /= 2

# lonlat1 = np.array([[0,10],
#            [0,20],
#            [0,30],
#            [0,40],
#            [0,50],
#            [10,0],
#            [20,0],
#            [30,0],
#            [40,0],
#            [50,0]])
# lonlat2 = np.array([0,0]).reshape(1,-1)

print(lonlat1)
print(lonlat2)


[[-82.92889435  47.38642061]
 [ 55.76141368 -72.6758314 ]
 [  8.02975854 -33.7407885 ]
 ...
 [-57.08506214 -15.70052389]
 [142.52639828 -80.64144282]
 [-98.21787179 -55.59724143]]
[[-138.9828529     3.51510725]]


In [192]:
%%timeit
# Measure execution time for scikit-learn haversine_distances
# start_time = time.time()
dists1 = haversine_distances(np.radians(lonlat1), np.radians(lonlat2))
# sklearn_time = time.time() - start_time


1.44 ms ± 318 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [193]:
%%timeit
# Measure execution time for optimized haversine_distances
# start_time = time.time()
dists2 = optimized_haversine_distances(np.radians(lonlat1), np.radians(lonlat2))
# optimized_time = time.time() - start_time

# print(f"Execution time for scikit-learn haversine_distances: {sklearn_time:.6f} seconds")
# print(f"Execution time for optimized haversine_distances: {optimized_time:.6f} seconds")


/var/folders/m9/32v45xkn4zx6js6htb1prb2r0000gn/T/ipykernel_9906/836437542.py:25: RuntimeWarning: invalid value encountered in sqrt
  distances = 2 * np.arcsin(np.sqrt(a))


1.05 ms ± 273 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [194]:
print(lonlat1)
print(lonlat2)
latlon1 = lonlat1[:,::-1]
latlon2 = lonlat2[:,::-1]


[[-82.92889435  47.38642061]
 [ 55.76141368 -72.6758314 ]
 [  8.02975854 -33.7407885 ]
 ...
 [-57.08506214 -15.70052389]
 [142.52639828 -80.64144282]
 [-98.21787179 -55.59724143]]
[[-138.9828529     3.51510725]]


In [195]:
%%timeit
dist3 = hvs.haversine_vector(latlon2,latlon1,comb=True,unit='rad')

1.23 ms ± 280 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [196]:
%%timeit
# print(latlon2)
# print(latlon1)
dist4 = np.array([slow_haversine_distances(np.radians(x), np.radians(latlon2)) for x in latlon1])


341 ms ± 155 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [199]:
%%timeit
dist5 = haversine_distances(np.radians(latlon1),np.radians(latlon2))
# dist4 = slow_haversine_distances(latlon2,latlon1)

1.19 ms ± 49 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [198]:
# print(lonlat1)
# print(lonlat2)
# print(latlon1)
# print(latlon2)
print(dists1)
print(dists2)
print(dist3)
print(dist4)
print(dist5)

[[1.32023731]
 [1.80521732]
 [2.04822275]
 ...
 [2.52849498]
 [1.36302152]
 [2.01045912]]
[       nan        nan 0.83456533 ...        nan 1.42586084 0.03494624]
[[2.6383627 ]
 [1.63633826]
 [1.80885406]
 ...
 [1.20221296]
 [1.35306881]
 [0.44263782]]
[[2.6383627 ]
 [1.63633826]
 [1.80885406]
 ...
 [1.20221296]
 [1.35306881]
 [0.44263782]]
[[2.6383627 ]
 [1.63633826]
 [1.80885406]
 ...
 [1.20221296]
 [1.35306881]
 [0.44263782]]
